In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('./Data/FakeNewsData.csv')

df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [3]:
# Drop Nan values
df = df.dropna()

In [4]:
# Get independent features
X = df.drop('label', axis = 1)

In [5]:
# Get dependent features
y = df['label']

In [6]:
X.shape

(18285, 4)

In [7]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM,Dense

In [8]:
# Vocabulary size
vocab_size = 10000

## Onehot Representation

In [9]:
messages = X.copy()
messages.reset_index(inplace = True)

messages.head()

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [10]:
import nltk
import re
from nltk.corpus import stopwords

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ciya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
# Data Preprocessing
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
corpus = []
for i in range(len(messages)):
    review = re.sub('[^a-zA-Z]',' ',messages['text'][i])
    review = review.lower().split()
    review = [lemmatizer.lemmatize(word) for word in review if word not in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [12]:
onehot_repr = [one_hot(words,vocab_size) for words in corpus]

In [19]:
len(corpus[0].split())

464

In [22]:
len(onehot_repr[0])

469

In [ ]:
(onehot_repr[0])

In [14]:
len(corpus), len(onehot_repr)

(18285, 18285)

In [15]:
max_len = 0
txt = ''
for i in range(len(corpus)):
    if len(corpus[i]) > max_len:
        max_len = len(corpus[i])
        txt = corpus[i]
max_len

142961

In [16]:
max_len = 0
txt = ''
for i in range(len(onehot_repr)):
    if len(onehot_repr[i]) > max_len:
        max_len = len(onehot_repr[i])
        txt = onehot_repr[i]
max_len

21002

## Embedding Representation

In [ ]:
sent_length = max_len
embedded_docs = pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)

In [ ]:
embedded_docs[78]

In [ ]:
# Creating the model
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(input_dim = vocab_size,
                    output_dim = embedding_vector_features,
                    input_length = sent_length))
model.add(LSTM(100))
model.add(Dense(1,
                activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])
print(model.summary())

In [ ]:
len(embedded_docs), y.shape

In [ ]:
import numpy as np
from scipy import sparse
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
"""# Convert your dense array - embedded_docs into a sparse matrix
X_final = sparse.csr_matrix(embedded_docs)

# Convert targt labels to sparse matrix if needed, but it seems it's not needed in here
y_final = sparse.csr_matrix(y)"""

In [ ]:
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [ ]:
X_final.shape, y_final.shape, y.shape

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X_final,y_final,test_size=0.33,random_state=42)

In [ ]:
"""# Convert data into sparse format to save memory
X_train_sparse = sparse.csr_matrix(X_train)
X_test_sparse = sparse.csr_matrix(X_test)"""

In [ ]:
"""X_train_sparse.shape, X_train.shape, X_test_sparse.shape, X_test.shape  
## It's not changed after sparse matrix application since it's been already applied"""

In [ ]:
"""# Define data generator for sparse data
def data_generator(X_sparse, y, batch_size):
    dataset_size = X_sparse.shape[0]
    while True:
        for i in range(0,dataset_size,batch_size):
            X_batch = X_sparse[i:i+batch_size].toarray() # Convert sparse to dense on the fly
            y_batch = y[i:i+batch_size]
            yield X_batch,y_batch"""

## Model Training

In [ ]:
"""# Batch size for training
batch_size = 32

# Create data generators for training and validation
train_generator = data_generator(X_train_sparse,y_train,batch_size)
validation_generator = data_generator(X_test_sparse,y_test,batch_size)"""

In [ ]:
"""# Use early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor = 'val_loss',patience = 3)

# Train the model with data generators
steps_per_epoch = X_train_sparse.shape[0] // batch_size
validation_steps = X_test_sparse.shape[0] // batch_size"""

In [ ]:
"""# Train the model
with tf.device('/GPU:0'):  # Explicitly run on GPU
    model.fit(train_generator,
              validation_data = validation_generator,
              steps_per_epoch = steps_per_epoch,
              validation_steps = validation_steps,
              epochs = 10,
              callbacks = [early_stopping],
              verbose = 1)"""

In [ ]:
# Train the model 
with tf.device('/GPU:0'):  # Explicitly run on GPU
    model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

## Adding Dropout

In [ ]:
from tensorflow.keras.layers import Dropout

# Creating the model
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(input_dim = voc_size,
                    output_dim = embedding_vector_features,
                    input_length = sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])
print(model.summary())

## Train the model after regularization

In [ ]:
# Train the model 
with tf.device('/GPU:0'):  # Explicitly run on GPU
    model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

## Performace Metrics and Accuracy

In [ ]:
y_pred = model.predict_classes(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [ ]:
confusion_matrix(y_test,y_pred)

In [ ]:
accuracy_score(y_test,y_pred)